# Классификация текстов: спам-фильтр для SMS

В этом задании вам предстоит **взять открытый датасет с SMS-сообщениями**, размеченными на спам ("spam") и не спам ("ham"), **построить на нем классификатор** текстов на эти два класса, **оценить его качество** с помощью кросс-валидации, **протестировать его работу** на отдельных примерах, и посмотреть, что будет происходить с качеством, если **менять параметры** вашей модели.

In [1]:
import pandas as pd
import numpy as np

In [92]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score

## 1. Загрузите датасет. 
Описание датасета можно посмотреть <a href="http://www.dt.fee.unicamp.br/~tiago/smsspamcollection">здесь</a>

датасет см. здесь: "data/SMSSpamCollection.txt"

## 2. Считайте датасет в Python 
(можете сразу грузить все в память, выборка небольшая), выясните, что используется в качестве разделителей и как проставляются метки классов.

In [43]:
data = pd.read_csv('data/SMSSpamCollection.txt', sep='\t', header=None, names=["label", "text"])

In [44]:
data.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


## 3. Готовим данные
Подготовьте для дальнейшей работы два списка: 
список текстов в порядке их следования в датасете и список соответствующих им меток классов. В качестве метки класса используйте 1 для спама и 0 для "не спама".

In [45]:
data['label'] = data['label'].map({'spam':1, 'ham':0})

In [64]:
labels = list(data['label'])
texts = list(data['text'])

## 4. Получаем матрицу признаков
Используя sklearn.feature_extraction.text.CountVectorizer со стандартными настройками, получите из списка текстов матрицу признаков X.

In [67]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(texts)

## 5. оценка качества бейзлайна
Оцените качество классификации текстов с помощью LogisticRegression() с параметрами по умолчанию, используя sklearn.cross_validation.cross_val_score (model_selection) и посчитав среднее арифметическое качества на отдельных fold'ах. Установите random_state=2. Параметр cv задайте равным 10. В качестве метрики качества используйте f1-меру. Получившееся качество - один из ответов, которые потребуются при сдаче задания. Ответ округлить до 1 знака после запятой.

**подготовим функцию для записи ответов**

In [71]:
def write_answer(number, answer):
    with open('answers/q{}.txt'.format(number), 'w') as file:
        file.write(str(answer))

In [69]:
classifier = LogisticRegression(random_state=2)
score = cross_val_score(classifier, X, labels, cv=10, n_jobs=-1, scoring='f1').mean()
score

0.9326402983610631

In [72]:
write_answer(5, round(score, 1))

## 6. тест классификатора на отдельных примерах
А теперь обучите классификатор на всей выборке и спрогнозируйте с его помощью класс для следующих сообщений:

**"FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB"**

**"FreeMsg: Txt: claim your reward of 3 hours talk time"**

**"Have you visited the last lecture on physics?"**

**"Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$"**

**"Only 99$"**

Прогнозы классификатора (0 - не спам, 1 - спам), записанные через пробел, будут ответом в одном из вопросов ниже.

In [73]:
%%time
classifier = LogisticRegression(random_state=2)
classifier.fit(X, labels)

Wall time: 89.7 ms


In [75]:
messages = ["FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB",
            "FreeMsg: Txt: claim your reward of 3 hours talk time", 
            "Have you visited the last lecture on physics?", 
            "Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$", 
            "Only 99$"]
predict = classifier.predict(vectorizer.transform(messages))
predict

array([1, 1, 0, 0, 0], dtype=int64)

In [82]:
write_answer(6, ' '.join(predict.astype(str)))

## 7. эксперименты с n-граммами и логистической регрессией
Задайте в CountVectorizer параметр ngram_range=(2,2), затем ngram_range=(3,3), затем ngram_range=(1,3). Во всех трех случаях измерьте получившееся в кросс-валидации значение f1-меры, округлите до второго знака после точки, и выпишете результаты через пробел в том же порядке. В данном эксперименте мы пробовали добавлять в признаки n-граммы для разных диапазонов n - только биграммы, только триграммы, и, наконец, все вместе - униграммы, биграммы и триграммы. Обратите внимание, что статистики по биграммам и триграммам намного меньше, поэтому классификатор только на них работает хуже. В то же время это не ухудшает результат сколько-нибудь существенно, если добавлять их вместе с униграммами, т.к. за счет регуляризации линейный классификатор не склонен сильно переобучаться на этих признаках.

In [83]:
%%time
scores = []
ranges = [(2,2), (3,3), (1,3)]
for rang in ranges:
    vectorizer = CountVectorizer(ngram_range=rang)
    X = vectorizer.fit_transform(texts)
    classifier = LogisticRegression(random_state=2)
    score = cross_val_score(classifier, X, labels, cv=10, n_jobs=-1, scoring='f1').mean()
    scores.append(str(round(score, 2)))

Wall time: 1min 52s


In [85]:
write_answer(7, ' '.join(scores))

## 8. эксперименты с n-граммами и наивным Байесом
Повторите аналогичный п.7 эксперимент, используя вместо логистической регрессии MultinomialNB(). Обратите внимание, насколько сильнее (по сравнению с линейным классификатором) наивный Байес страдает от нехватки статистики по биграммам и триграммам.

In [90]:
%%time
scores = []
ranges = [(2,2), (3,3), (1,3)]
for rang in ranges:
    vectorizer = CountVectorizer(ngram_range=rang)
    X = vectorizer.fit_transform(texts)
    classifier = MultinomialNB()
    score = cross_val_score(classifier, X, labels, cv=10, n_jobs=-1, scoring='f1').mean()
    scores.append(str(round(score, 2)))

Wall time: 1min 12s


In [91]:
write_answer(8, ' '.join(scores))

## 9. эксперимент с tf*idf на униграммах и логистической регрессией
Попробуйте использовать в логистической регрессии в качестве признаков $Tf*idf$ из $TfidfVectorizer$ на униграммах. Повысилось или понизилось качество на кросс-валидации по сравнению с CountVectorizer на униграммах? (напишите в файле с ответом 1, если повысилось, -1, если понизилось, и 0, если изменилось не более чем на 0.01). Обратите внимание, что результат перехода к $tf*idf$ не всегда будет таким - если вы наблюдаете какое-то явление на одном датасете, не надо сразу же его обобщать на любые данные.

In [94]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(texts)
classifier = LogisticRegression(random_state=2)
score = cross_val_score(classifier, X, labels, cv=10, n_jobs=-1, scoring='f1').mean()
score

0.85285995541724557

**Ответ:** качество на кросс-валидации по сравнению с CountVectorizer на униграммах - **понизилось** (ответ = **- 1**)

In [95]:
write_answer(9, -1)